In [ ]:
# import libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import math
import json
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from sklearn.externals import joblib
import scipy.sparse
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
import warnings; warnings.simplefilter('ignore')
%matplotlib inline

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

for dirname, _, filenames in os.walk('/kaggle/working/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# Load the Dataset and Add headers

## load appliances user, item meta, train/test/val/ split data

In [ ]:
# Appliances=pd.read_json('/kaggle/input/meta-appliances-jsonl/Appliances.jsonl', lines=True)
# meta_Appliances=pd.read_json('/kaggle/input/meta-appliances-jsonl/meta_Appliances.jsonl', lines=True)
# application_train_split=pd.read_csv('/kaggle/input/meta-appliances-jsonl/application_train_split.csv')

## remove na/null rows

In [ ]:

# application_train_split = application_train_split.dropna()
# application_train_split = application_train_split[application_train_split['history'].notnull()]
# # application_train_hist_split = application_train_split.groupby('user_id')['history'].apply(list).reset_index()
# application_train_split.head(2)
# len(application_train_split['user_id'])
# len(application_train_split['user_id'].unique())

## filter train set based on history length(past purchased items list)

In [ ]:
# application_train_split['history_len'] = application_train_split['history'].apply(len)
# application_train_his_sum_split = application_train_split.groupby('user_id')['history_len'].sum().reset_index()


# application_train_his_sum_split = application_train_his_sum_split[application_train_his_sum_split['history_len'] >= 23]
# application_train_his_sum_split = application_train_his_sum_split[application_train_his_sum_split['history_len'] <= 75]
# application_train_his_sum_split.head(2)
# application_train_his_sum_split.describe()['history_len']
# len(application_train_his_sum_split)
# len(application_train_his_sum_split['user_id'].unique())

# application_train_split = application_train_split[application_train_split['user_id'].isin(application_train_his_sum_split['user_id'].unique())]
# len(application_train_split)
# len(application_train_split['user_id'].unique())

## filter test set based on history length(past purchased items list)

In [ ]:
# application_test_split=pd.read_csv('/kaggle/input/meta-appliances-jsonl/application_test_split.csv')
# # application_test_split = application_test_split[application_test_split['user_id'].isin(grouped_sum1['user_id'])]
# application_test_split = application_test_split.dropna()
# application_test_split = application_test_split[application_test_split['history'].notnull()]
# application_test_split = application_test_split[application_test_split['user_id'].isin(application_train_his_sum_split['user_id'].unique())]
# application_test_hist_split = application_test_split.groupby('user_id')['history'].apply(list).reset_index()
# application_test_hist_split['history_len'] = application_test_hist_split['history'].apply(len)
# application_test_hist_sum_split = application_test_hist_split.groupby('user_id')['history_len'].sum().reset_index()
# # application_test_hist_sum_split = application_test_hist_split.groupby('user_id')['history_len'].sum().reset_index(drop=False)

# application_test_hist_sum_split.describe()['history_len']
# application_test_hist_sum_split.head(2)
# application_test_split.head(2)
# len(application_test_split['user_id'])
# len(application_test_split['user_id'].unique())

In [ ]:
# application_test_split.index

In [ ]:
# # union_user_ids
# # union_user_ids = pd.Index(application_train_his_sum_split['user_id']).union(application_test_hist_sum_split['user_id'])
# # Appliances_new = Appliances.loc[sub_df.index]
# Appliances_new =  Appliances[Appliances['user_id'].isin(application_train_his_sum_split['user_id'].unique())]
# Appliances_new.head(2)
# meta_Appliances_new = meta_Appliances[meta_Appliances['parent_asin'].isin(Appliances_new['parent_asin'].unique()) ]

# meta_Appliances_new.head(2)
# len(Appliances_new['user_id'])
# len(Appliances_new['user_id'].unique())

In [ ]:
# result = Appliances_new[Appliances_new['verified_purchase']].groupby('user_id').size().reset_index(name='purchased_count')
# result[result['purchased_count']>4].describe()['purchased_count']
# result.head(2)

## merge user review & item meta files

In [ ]:
# # drop 'images','videos', 'bought_together', 'subtitle', 'author', and delete clomuns main_category is none, price is none 
# meta_Appliances_new = meta_Appliances_new.drop(columns=['images','videos', 'bought_together', 'subtitle', 'author'])
# meta_Appliances_new = meta_Appliances_new[meta_Appliances_new['main_category'].notnull()]
# meta_Appliances_new = meta_Appliances_new[meta_Appliances_new['price'].notnull()]
# meta_Appliances_new = meta_Appliances_new[meta_Appliances_new['store'].notnull()]


# # 同一个parent_asin，图同颜色或款式单独一列
# print("origin user_id num: {}, origin parent_asin num:{}".format(len(Appliances_new['user_id']), len(meta_Appliances_new['parent_asin'])))
# print("unique origin user_id num: {}, unique origin parent_asin num:{}".format(Appliances_new['user_id'].unique().size, meta_Appliances_new['parent_asin'].unique().size))
# print("new user_id num: {}, new parent_asin num:{}".format(len(Appliances_new['user_id']), len(meta_Appliances_new['parent_asin'])))
# print("unique new user_id num: {}, unique new parent_asin num:{}".format(Appliances_new['user_id'].unique().size, meta_Appliances_new['parent_asin'].unique().size))
# #  merge two data frame
# Appliances_meta_merge_train = application_train_split.merge(meta_Appliances_new, on='parent_asin', how='inner')
# Appliances_meta_merge_test = application_test_split.merge(meta_Appliances_new, on='parent_asin', how='inner')
# Appliances_meta_merge_train.head(2)
# Appliances_meta_merge_test.head(2)
# print("merged_df len:{}".format(Appliances_meta_merge_train['user_id'].size))
# Appliances_meta_merge_train.to_csv('/kaggle/working/Appliances_meta_merge_train.csv')
# Appliances_meta_merge_test.to_csv('/kaggle/working/Appliances_meta_merge_test.csv')
# meta_Appliances_new.to_csv('/kaggle/working/meta_Appliances_new2.csv')

## filter train/test set based on rate count (abandon)

In [ ]:
# if os.path.exists('/kaggle/input/meta-appliances-jsonl/Appliances_review_meta_merge.csv'):
# # if 0:
#     Appliances_review_meta_merge = pd.read_csv('/kaggle/input/meta-appliances-jsonl/Appliances_review_meta_merge.csv')
#     meta_Appliances_new = pd.read_csv('/kaggle/input/meta-appliances-jsonl/meta_Appliances_new.csv')
#     Appliances_review_meta_merge = Appliances_review_meta_merge[Appliances_review_meta_merge['title_x'].notnull()]
#     Appliances_review_meta_merge = Appliances_review_meta_merge[Appliances_review_meta_merge['text'].notnull()]
#     # Appliances_review_meta_merge = Appliances_review_meta_merge.drop(columns=['Unnamed: 0'])
#     # Appliances_review_meta_merge = Appliances_review_meta_merge.groupby(['user_id', 'parent_asin']).agg({'rating': 'mean', 'title_x': 'first', 
#     #     'text': 'first', 'asin': 'first', 'timestamp': 'first', 'main_category': 'first', 'title_y': 'first', 'average_rating': 'first',
#     #    'rating_number': 'first', 'features': 'first', 'description': 'first', 'price': 'first', 'store': 'first',
#     #    'categories': 'first', 'details': 'first'}).reset_index()
# else:
#     # Drop useless columns & Handling Missing values
#     Appliances=pd.read_json('/kaggle/input/meta-appliances-jsonl/Appliances.jsonl', lines=True)
#     meta_Appliances=pd.read_json('/kaggle/input/meta-appliances-jsonl/meta_Appliances.jsonl', lines=True)
#     # drop image, asin, use parent_asin as unique id
#     Appliances = Appliances.drop(columns=['images'])
#     # Appliances.head()
#     # drop 'images','videos', 'bought_together', 'subtitle', 'author', and delete clomuns main_category is none, price is none 
#     meta_Appliances = meta_Appliances.drop(columns=['images','videos', 'bought_together', 'subtitle', 'author'])
#     meta_Appliances = meta_Appliances[meta_Appliances['main_category'].notnull()]
#     meta_Appliances = meta_Appliances[meta_Appliances['price'].notnull()]
#     meta_Appliances = meta_Appliances[meta_Appliances['store'].notnull()]
#     # meta_Appliances.head()
#     # 数据太多了，notobook memory 不够，截取前评价数据最多的5w user data, 并且把与5w user 有交互的product的meta信息筛选出来
#     Ranked_user_review_num = Appliances.groupby(by='user_id')['rating'].count().sort_values(ascending=False)
#     # Ranked_user_review_num[:5000]
#     Appliances_new = Appliances[Appliances['user_id'].isin(Ranked_user_review_num[:5000].index.tolist())]
#     Appliances_new_parent_asin = Appliances_new['parent_asin'].unique()
#     meta_Appliances_new = meta_Appliances[meta_Appliances['parent_asin'].isin(Appliances_new_parent_asin) ]
#     # 同一个parent_asin，图同颜色或款式单独一列
#     print("origin user_id num: {}, origin parent_asin num:{}".format(len(Appliances['user_id']), len(meta_Appliances['parent_asin'])))
#     print("unique origin user_id num: {}, unique origin parent_asin num:{}".format(Appliances['user_id'].unique().size, meta_Appliances['parent_asin'].unique().size))
#     print("new user_id num: {}, new parent_asin num:{}".format(len(Appliances_new['user_id']), len(meta_Appliances_new['parent_asin'])))
#     print("unique new user_id num: {}, unique new parent_asin num:{}".format(Appliances_new['user_id'].unique().size, meta_Appliances_new['parent_asin'].unique().size))
#     #  merge two data frame
#     Appliances_review_meta_merge = Appliances_new.merge(meta_Appliances_new, on='parent_asin', how='inner')
#     Appliances_review_meta_merge.head(2)
#     print("merged_df len:{}".format(Appliances_review_meta_merge.size))
#     Appliances_review_meta_merge.to_csv('/kaggle/working/Appliances_review_meta_merge2.csv')
#     meta_Appliances_new.to_csv('/kaggle/working/meta_Appliances_new.csv')

In [ ]:
Appliances_meta_merge_train = pd.read_csv('/kaggle/input/meta-appliances-jsonl/Appliances_meta_merge_train.csv')
Appliances_meta_merge_test = pd.read_csv('/kaggle/input/meta-appliances-jsonl/Appliances_meta_merge_test.csv')
meta_Appliances_new= pd.read_csv('/kaggle/input/meta-appliances-jsonl/meta_Appliances_new2.csv')
application_train_split= pd.read_csv('/kaggle/input/meta-appliances-jsonl/application_train_split_custom.csv')
application_test_split= pd.read_csv('/kaggle/input/meta-appliances-jsonl/application_test_split_custom.csv')

# visual data attribute

In [ ]:
# 查看user，product 数量，以及数据的sparsity
len(Appliances_meta_merge_train)
user_id_unique = Appliances_meta_merge_train['user_id'].unique()
product_id_unique = Appliances_meta_merge_train['parent_asin'].unique()
print("user_id_unique:{},product_id_unique:{}".format(len(user_id_unique), len(product_id_unique)))
user_id_map = pd.DataFrame({'user_id':user_id_unique, 'mapped_user_id':np.arange(len(user_id_unique))})
product_id_map = pd.DataFrame({'parent_asin':product_id_unique, 'mapped_parent_asin':np.arange(len(product_id_unique))})
# Total possible interactions
total_possible_interactions = len(user_id_unique) * len(product_id_unique)
# Actual interactions
actual_interactions = Appliances_meta_merge_train.shape[0]
# Calculate sparsity
sparsity = 1 - (actual_interactions / total_possible_interactions)
print(f'Total possible interactions: {total_possible_interactions}')
print(f'Actual interactions: {actual_interactions}')
print(f'Sparsity: {sparsity:.2%}')

In [ ]:
# encode user_id and parent_asin to unique id
# Appliances_review_meta_merge = Appliances_review_meta_merge.merge(user_id_map, on='user_id', how='left').drop(columns=['user_id']).rename(columns={'mapped_user_id': 'user_id'})
# Appliances_review_meta_merge = Appliances_review_meta_merge.merge(product_id_map, on='parent_asin', how='left').drop(columns=['parent_asin']).rename(columns={'mapped_parent_asin': 'parent_asin'})
# Appliances_review_meta_merge.head(2)
# len(Appliances_review_meta_merge['user_id'].unique())
# len(Appliances_review_meta_merge['parent_asin'].unique())

**查看数据属性：rating 统计， 范围，空数据**

In [ ]:
#Five point summary 
Appliances_meta_merge_train.describe()['rating'].T

In [ ]:
#Find the minimum and maximum ratings
print('Minimum rating is: %d' %(Appliances_meta_merge_train.rating.min()))
print('Maximum rating is: %d' %(Appliances_meta_merge_train.rating.max()))

The rating of the product range from 0 to 1

In [ ]:
#Check for missing values
print('Number of missing values across columns: \n',Appliances_meta_merge_train.isnull().sum())

## Ratings

In [ ]:
# Check the distribution of the rating
with sns.axes_style('white'):
    g = sns.factorplot("rating", data=Appliances_meta_merge_train, aspect=2.0,kind='count')
    g.set_ylabels("Total number of ratings")

Most of the people has given the rating of 5

## Unique Users and products


In [ ]:
print("Total data ")
print("-"*50)
print("\nTotal no of ratings :",Appliances_meta_merge_train.shape[0])
print("Total No of Users   :", len(np.unique(Appliances_meta_merge_train.user_id)))
print("Total No of products  :", len(np.unique(Appliances_meta_merge_train.parent_asin)))

## Analyzing the rating

In [ ]:
#Analysis of rating given by the user 

no_of_rated_products_per_user = Appliances_meta_merge_train.groupby(by='user_id')['rating'].count().sort_values(ascending=False)

no_of_rated_products_per_user.head()

In [ ]:
no_of_rated_products_per_user.describe()

In [ ]:
quantiles = no_of_rated_products_per_user.quantile(np.arange(0,1.01,0.01), interpolation='higher')


In [ ]:
plt.figure(figsize=(10,10))
plt.title("Quantiles and their Values")
quantiles.plot()
# quantiles with 0.05 difference
plt.scatter(x=quantiles.index[::5], y=quantiles.values[::5], c='orange', label="quantiles with 0.05 intervals")
# quantiles with 0.25 difference
plt.scatter(x=quantiles.index[::25], y=quantiles.values[::25], c='m', label = "quantiles with 0.25 intervals")
plt.ylabel('No of ratings by user')
plt.xlabel('Value at the quantile')
plt.legend(loc='best')
plt.show()

In [ ]:
# new_df_collabrative = Appliances_meta_merge_train.drop(columns=[ 'title_x','text', 'asin', 'helpful_vote', 'verified_purchase', 'main_category', 'title_y', 'average_rating', 'rating_number'])
# new_df_collabrative = new_df_collabrative.drop(columns=['features', 'description', 'price', 'store', 'categories', 'details','timestamp'])
# new_df_collabrative = new_df_collabrative.groupby(['user_id', 'parent_asin']).agg({'rating': 'mean'}).reset_index()

# Collaberative filtering (Item-Item recommedation)

Collaborative filtering is commonly used for recommender systems. These techniques aim to fill in the missing entries of a user-item association matrix. We are going to use collaborative filtering (CF) approach.
CF is based on the idea that the best recommendations come from people who have similar tastes. In other words, it uses historical item ratings of like-minded people to predict how someone would rate an item.Collaborative filtering has two sub-categories that are generally called memory based and model-based approaches.



In [ ]:
from surprise import KNNWithMeans
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
import os
from surprise.model_selection import train_test_split
from surprise import SVD
from surprise import accuracy

In [ ]:
Appliances_meta_merge_train.head(2)

In [ ]:

Appliances_meta_merge_train.head(2)

# Convert the timestamp to a datetime format
Appliances_meta_merge_train['timestamp'] = pd.to_datetime(Appliances_meta_merge_train['timestamp'])

# plot hist of timestamp
plt.figure(figsize=(12, 6))
plt.hist(Appliances_meta_merge_train['timestamp'], bins=20, edgecolor='black', alpha=0.7)
plt.title('Distribution of Timestamps')
plt.xlabel('Timestamp')
plt.ylabel('Frequency')
plt.show()

## split dataset to train & test

### split tran/test based on fixed timestamp(abandon)

In [ ]:
# # 1. split train & test dataset based on timestamp
# split_timestamp = pd.Timestamp('2022-03-01 00:00:00')
# train_df = Appliances_review_meta_merge[Appliances_review_meta_merge['timestamp'] <= split_timestamp]
# test_df = Appliances_review_meta_merge[Appliances_review_meta_merge['timestamp'] > split_timestamp]
# print("train len: {}, test len:{}".format(len(train_df), len(test_df)))
# # Load the training dataset
# reader = Reader(rating_scale=(1, 5), line_format='user item rating', sep=',')
# train_data = Dataset.load_from_df(train_df[['user_id', 'parent_asin', 'rating']], reader)
# # Build the full training set
# trainset = train_data.build_full_trainset()
# # Convert the test set to a list of tuples (user_id, item_id, rating)
# testset = list(test_df[['user_id', 'parent_asin', 'rating']].itertuples(index=False, name=None))
# # # 2. split train & test dataset randomly
# # reader = Reader(rating_scale=(1, 5), line_format='user item rating', sep=',')
# # data = Dataset.load_from_df(new_df_collabrative[['user_id', 'parent_asin', 'rating']],reader)
# # trainset, testset = train_test_split(data, test_size=0.1,random_state=10)

### train& test dataset based on predefined timestamp

In [ ]:
# union_user_ids = pd.Index(application_train_his_sum_split['user_id']).union(application_test_hist_sum_split['user_id'])
# 1. split train & test dataset based on timestamp predefined
# Appliances_review_meta_merge = Appliances_review_meta_merge.loc[sub_df.index]
train_df = Appliances_meta_merge_train
test_df = Appliances_meta_merge_test
print("train len: {}, test len:{}".format(len(train_df), len(test_df)))
# Load the training dataset
reader = Reader(rating_scale=(1, 5), line_format='user item rating', sep=',')
train_data = Dataset.load_from_df(train_df[['user_id', 'parent_asin', 'rating']], reader)
# Build the full training set
trainset = train_data.build_full_trainset()
# Convert the test set to a list of tuples (user_id, item_id, rating)
testset = list(test_df[['user_id', 'parent_asin', 'rating']].itertuples(index=False, name=None))

In [ ]:
# test_df.groupby()
# test_df_group = test_df.groupby('user_id')['parent_asin'].apply(list).reset_index()
train_df_group = train_df.groupby('user_id').apply(lambda x: pd.Series({
    'parent_asin': x['parent_asin'].tolist(),
    'rating': x['rating'].tolist()
})).reset_index()
len(train_df_group['user_id'])
len(train_df_group['user_id'].unique())
train_df_group.head()
# train_df_group = list(train_df_group[['user_id', 'parent_asin', 'rating']].itertuples(index=False, name=None))

In [ ]:
# testset_group[0]
# train_df_group[train_df_group['user_id']==testset_group[0][0]]

In [ ]:
# test_df.groupby()
# test_df_group = test_df.groupby('user_id')['parent_asin'].apply(list).reset_index()
test_df_group = test_df.groupby('user_id').apply(lambda x: pd.Series({
    'parent_asin': x['parent_asin'].tolist(),
    'rating': x['rating'].tolist()
})).reset_index()
len(test_df_group['user_id'])
len(test_df_group['user_id'].unique())
test_df_group.head()
testset_group = list(test_df_group[['user_id', 'parent_asin', 'rating']].itertuples(index=False, name=None))

In [ ]:
# # Create user-item matrix
# user_item_matrix = train_df.pivot(index='user_id', columns='parent_asin', values='rating')
# # Fill missing values (NaN) with zeros
# user_item_matrix_filled = user_item_matrix.fillna(0)
# # Transform the matrix back to a DataFrame in long format
# df_long = user_item_matrix_filled.stack().reset_index()
# df_long.columns = ['user_id', 'item_id', 'rating']
# user_item_train_data = Dataset.load_from_df(df_long, reader)
# trainset = user_item_train_data.build_full_trainset()

## evaluation metrics

In [ ]:
def dcg_at_k(recommended_items, relevant_items, k):
    dcg = 0.0
    for i, item in enumerate(recommended_items[:k]):
        if item in relevant_items:
            dcg += 1.0 / math.log2(i + 2)
    return dcg

def idcg_at_k(relevant_items, k):
    idcg = 0.0
    for i in range(min(len(relevant_items), k)):
        idcg += 1.0 / math.log2(i + 2)
    return idcg

def ndcg_at_k(recommended_items, relevant_items, k):
    recommended_items_relevant = recommended_items[:len(relevant_items)]
    recommended_items_add = [item for item in recommended_items if ((item in relevant_items) and (item not in recommended_items_relevant))]
    recommended_items_relevant.extend(recommended_items_add)
    if(len(recommended_items_relevant) > len(relevant_items)):
        count = len(recommended_items_relevant) - len(relevant_items)
        for i in range(len(recommended_items_relevant) - 1, -1, -1):
            if recommended_items_relevant[i] not in relevant_items:
                recommended_items_relevant.pop(i)
                count += 1
                if count == 2:
                    break
 
    
    # print(recommended_items_relevant)
    idcg = idcg_at_k(relevant_items, k)
    if not idcg:
        return 0.0
    dcg = dcg_at_k(recommended_items_relevant, relevant_items, k)
    return dcg / idcg



In [ ]:
# evaluation metrics
# Get recommendations for the users in the test set
from collections import defaultdict

# Create a dict to store the ratings in the test set
test_dict = defaultdict(set)
for uid, iid, true_r in testset:
    test_df_user_item = test_df[test_df['user_id']==uid]
    test_df_purchased_item = test_df_user_item['parent_asin'].unique()
    test_df_purchased_list = [item for item in test_df_purchased_item]
    test_dict[uid] = test_df_purchased_list

test_dict['AFTEMMCIMU62ONZHLLXEXSME4Z6Q']
# Function to calculate precision, recall, and F1-score
def precision_recall_f1_at_k(top_n_recommendations, testset, k=3):
    precision_sum = 0
    recall_sum = 0
    f1_sum = 0
    total_users = 0 
    ndcg_sum = 0

    # Calculate Precision, Recall, and F1 for each user
    for user_id, recommended_items in top_n_recommendations.items():
        # if user_id in test_dict:
       
        seen = set()
        recommended_items_set = [x for x in recommended_items if not (x in seen or seen.add(x))]    
        recommended_items_set = recommended_items_set[:k]
        # print(recommended_items_set[:5])

        seen = set()
        relevant_items_set = [x for x in test_dict[user_id] if not (x in seen or seen.add(x))]    
        # print(relevant_items_set)
        
        # relevant_items_set = list(set([item for item in test_dict[user_id]]) )
        relevant_num = len(relevant_items_set)
        
        # Precision: how many recommended items are relevant
        hits = len(set(recommended_items_set) & set(relevant_items_set))
        precision = hits / k

        # Recall: how many relevant items are recommended
        recall = hits / len(relevant_items_set) if len(relevant_items_set) > 0 else 0

        # F1-Score: Harmonic mean of Precision and Recall
        if precision + recall > 0:
            f1_score = 2 * (precision * recall) / (precision + recall)
        else:
            f1_score = 0

        # Calculate NDCG
        ndcg_score = ndcg_at_k(list(recommended_items_set), list(relevant_items_set), k)

        precision_sum += precision
        recall_sum += recall
        f1_sum += f1_score
        ndcg_sum += ndcg_score
        total_users += 1
        

    # Calculate average precision, recall, and F1-score
    avg_precision = precision_sum / total_users
    avg_recall = recall_sum / total_users
    avg_f1_score = f1_sum / total_users
    avg_ndcg = ndcg_sum / total_users

    return avg_precision, avg_recall, avg_f1_score, avg_ndcg



In [ ]:
# # hybrid_recommendation['AFTEMMCIMU62ONZHLLXEXSME4Z6Q']
# hybrid_recommendation1 = {}
# hybrid_recommendation1['AFTEMMCIMU62ONZHLLXEXSME4Z6Q'] = hybrid_recommendation['AFTEMMCIMU62ONZHLLXEXSME4Z6Q']
# avg_precision_hybrid1, avg_recall_hybrid1, avg_f1_score_hybrid1, avg_ndcg_hybrid1 = precision_recall_f1_at_k(
#     hybrid_recommendation1, testset, k=100)

# hybrid_recommendation2 = {}
# hybrid_recommendation2['AFTEMMCIMU62ONZHLLXEXSME4Z6Q'] = top_n_recommendations_contentbased['AFTEMMCIMU62ONZHLLXEXSME4Z6Q']
# avg_precision_hybrid2, avg_recall_hybrid2, avg_f1_score_hybrid2, avg_ndcg_hybrid2 = precision_recall_f1_at_k(
#     hybrid_recommendation2, testset, k=100)

# len(hybrid_recommendation1.items())

# top_n_recommendations_contentbased['AFTEMMCIMU62ONZHLLXEXSME4Z6Q'][:5]

# hybrid_recommendation['AFTEMMCIMU62ONZHLLXEXSME4Z6Q'][:5]

In [ ]:
# avg_precision_hybrid, avg_recall_hybrid, avg_f1_score_hybrid, avg_ndcg_hybrid = precision_recall_f1_at_k(
#     hybrid_recommendation, testset, k=100)

In [ ]:
# Get all items in the dataset
train_df_items = list(set(Appliances_meta_merge_train['parent_asin'].unique()))
test_df_items = list(set(Appliances_meta_merge_test['parent_asin'].unique()))
all_items = set(train_df_items + test_df_items)
# all_items
len(train_df_items)
len(test_df_items)
len(all_items)

In [ ]:
# Get all items in the dataset
# train_df_items = Appliances_meta_merge_train['parent_asin'].unique()
# test_df_items = Appliances_meta_merge_test['parent_asin'].unique()
# all_items = set(Appliances_review_meta_merge['parent_asin'].unique())
from tqdm import tqdm
def get_recommendations_for_testset(testset_group, model, top_n=3):
    recommendations = {}
    # Iterate through each user in the test set
    # for user_id, _, true_rating in testset:
    #     recommendations[user_id] = []
    for index, user_item in enumerate(tqdm(testset, desc="Processing items")):
        # Get items the user has already rated in the test set
        user_id, purchased_items, true_rating = user_item
        rated_items = train_df[train_df['user_id']==user_id]['parent_asin'].unique()     
   
        # Get items the user has not rated yet
        unrated_items = all_items - set(rated_items)

        # Generate predictions for unrated items
        predictions = [model.predict(user_id, item) for item in unrated_items]

        # Sort predictions by predicted rating (highest first)
        sorted_predictions = sorted(predictions, key=lambda x: x.est, reverse=True)

        # Get the top N recommended items
        recommended_items = [pred.iid for pred in sorted_predictions[:top_n]]
        # print("user_id: {}, recommended_items:{}".format(user_id, recommended_items))

        recommendations[user_id] = list(recommended_items)
        # recommendations[user_id].extend(list(recommended_items))

    return recommendations


## user based collaborative filter

In [ ]:
# User-Based Collaborative Filtering
from surprise import KNNWithMeans
from surprise import accuracy
import random
seed = 42
np.random.seed(seed)
random.seed(seed)
TOP_K = 100
# eva_top_k = 100
eva_top_k = 100


In [ ]:

# User-based collaborative filtering
algo_user = KNNWithMeans(sim_options={'user_based': True})
algo_user.fit(trainset)

# Predict ratings for the testset
predictions_user = algo_user.test(testset)
# Evaluate the accuracy
RMSE = accuracy.rmse(predictions_user)
print("RMS for user based collaborative filter: {}".format(RMSE))

In [ ]:
# Top n recommendation 
userbased_recom_result_pth = "/kaggle/input/meta-appliances-jsonl/top_n_recommendations_userbased4.json"

if os.path.exists(userbased_recom_result_pth):
    top_n_recommendations_userbased = pd.read_json(userbased_recom_result_pth)
else:
    top_n_recommendations_userbased = get_recommendations_for_testset(testset, algo_user, top_n=TOP_K)
    with open('/kaggle/working/top_n_recommendations_userbased4.json', 'w') as f:
        json.dump(top_n_recommendations_userbased, f)

In [ ]:
# top_n_recommendations_userbased['AFYU5VCZB2GWHQ4T7EHLDZG7XG6Q']
testset_group[0]

In [ ]:
# Calculate precision, recall, and F1-score at K=10
eva_top_k = 100
avg_precision_userbased, avg_recall_userbased, avg_f1_score_userbased, avg_ndg_userbased = precision_recall_f1_at_k(top_n_recommendations_userbased, testset, k=eva_top_k)

print(f"Average Precision at K=3: {avg_precision_userbased:.4f}")
print(f"Average Recall at K=3: {avg_recall_userbased:.4f}")
print(f"Average F1-Score at K=3: {avg_f1_score_userbased:.4f}")
print(f"Average NDCG at K=3: {avg_ndg_userbased:.4f}")

## item based collaborative filter

In [ ]:
# Item-based collaborative filtering
algo_item = KNNWithMeans(sim_options={'user_based': False})
algo_item.fit(trainset)

# # 1. Predict ratings for the testset
predictions_item = algo_item.test(testset)
# Evaluate the accuracy
RMSE = accuracy.rmse(predictions_item)

print("RMS for item based collaborative filter: {}".format(RMSE))


In [ ]:
# Top n recommendation
itembased_recom_result_pth = "/kaggle/input/meta-appliances-jsonl/top_n_recommendations_itembased4.json"

if os.path.exists(itembased_recom_result_pth):
    top_n_recommendations_itembased = pd.read_json(itembased_recom_result_pth)
else:
    top_n_recommendations_itembased = get_recommendations_for_testset(testset, algo_item, top_n=TOP_K)
    with open('/kaggle/working/top_n_recommendations_itembased4.json', 'w') as f:
        json.dump(top_n_recommendations_itembased, f)

In [ ]:
# Calculate precision, recall, and F1-score at K=10
eva_top_k = 100
avg_precision_itembased, avg_recall_itembased, avg_f1_score_itembased, avg_ndg_itembased = precision_recall_f1_at_k(top_n_recommendations_itembased, testset, k=eva_top_k)

print(f"Average Precision at K=3: {avg_precision_itembased:.8f}")
print(f"Average Recall at K=3: {avg_recall_itembased:.8f}")
print(f"Average F1-Score at K=3: {avg_f1_score_itembased:.8f}")
print(f"Average NDG at K=3: {avg_ndg_itembased:.8f}")

## model based collaborative filter

In [ ]:
# Model-based collaborative filtering system
svd = SVD()
svd.fit(trainset)
# # Make predictions on the test set
# predictions = svd.test(testset)
# # Evaluate the model's performance using RMSE
# rmse = accuracy.rmse(predictions, verbose=True)
# print(f'RMSE: {rmse}')

# Get top-N recommendations for the users in the test set
predictions_svd = svd.test(testset)

# Evaluate the accuracy
RMSE = accuracy.rmse(predictions_svd)

print("RMS for item based collaborative filter: {}".format(RMSE))

In [ ]:
svd_model_based_recom_result_pth = "/kaggle/input/meta-appliances-jsonl/top_n_recommendations_svd4.json"

if os.path.exists(svd_model_based_recom_result_pth):
    top_n_recommendations_svd = pd.read_json(svd_model_based_recom_result_pth)
else:
    top_n_recommendations_svd = get_recommendations_for_testset(testset, svd, top_n=TOP_K)
    with open('/kaggle/working/top_n_recommendations_svd4.json', 'w') as f:
        json.dump(top_n_recommendations_svd, f)


In [ ]:
# Calculate precision, recall, and F1-score at K=3
eva_top_k=100
avg_precision_svd, avg_recall_svd, avg_f1_score_svd, avg_ndg_svd = precision_recall_f1_at_k(top_n_recommendations_svd, testset, k=eva_top_k)


print(f"Average Precision at K=3: {avg_precision_svd:.4f}")
print(f"Average Recall at K=3: {avg_recall_svd:.4f}")
print(f"Average F1-Score at K=3: {avg_f1_score_svd:.4f}")
print(f"Average NDG at K=3: {avg_ndg_svd:.4f}")

In [ ]:
# len(top_n_recommendations_svd)
# top_n_recommendations_svd['AFTEMMCIMU62ONZHLLXEXSME4Z6Q'][:2]

# content based method

In [ ]:
# content based recommendations
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
# Create a mapping from item_id to index
# Drop duplicates based on 'parent_asin' (keeping the first occurrence) & map parent_asin(item_id) to id(0-len)
meta_Appliances_new = meta_Appliances_new.drop_duplicates(subset=['parent_asin'])
meta_Appliances_new_reset = meta_Appliances_new.reset_index(drop=True)
meta_Appliances_new_reset.index = range(0, len(meta_Appliances_new_reset))
item_indices = pd.Series(meta_Appliances_new.index, index=meta_Appliances_new['parent_asin']).to_dict()
meta_Appliances_new_reset.head(2)
# item_indices

# # Combine text features
meta_Appliances_new['combined_features'] = meta_Appliances_new.apply(lambda x: f"{x['title']} {x['categories']} {x['description']} {x['details']}", axis=1)
meta_Appliances_new.head(2)
# Vectorize combined text features
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(meta_Appliances_new['combined_features'])

# Compute similarity matrix (cosine similarity)
cosine_sim_matrix = linear_kernel(tfidf_matrix, tfidf_matrix)



In [ ]:
len(train_df['parent_asin'].unique())
len(meta_Appliances_new_reset['parent_asin'].unique())
len(meta_Appliances_new[meta_Appliances_new['parent_asin'].isin(train_df['parent_asin'].unique())]['parent_asin'].unique())
len(train_df[train_df['parent_asin'].isin(meta_Appliances_new_reset['parent_asin'].unique())]['parent_asin'].unique())
len(test_df[test_df['parent_asin'].isin(meta_Appliances_new_reset['parent_asin'].unique())]['parent_asin'].unique())
# train_df['parent_asin'].unique() isin

In [ ]:
from surprise import Prediction

# Create a dict to store the ratings in the test set
test_dict_user_purchased = {}
for uid, iid, true_r in testset:
    test_df_user_item = train_df[train_df['user_id']==uid]
    test_df_purchased_item = test_df_user_item['parent_asin']
    test_df_purchased_item_list = [item for item in test_df_purchased_item]
    test_df_purchased_rating = test_df_user_item ['rating']
    test_df_purchased_rating_list = [item for item in test_df_purchased_rating.array]
    test_dict_user_purchased[uid] = {'item_id': test_df_purchased_item_list, 'rating':test_df_purchased_rating_list}

def get_user_profile(user_id, feat_matrix):
    user_interactions = test_dict_user_purchased[user_id]
    user_item_indices = [item_indices[item_id] for item_id in user_interactions['item_id']]
    user_item_ratings = user_interactions['rating']
    
    # Weighted average of item vectors
    # user_profile = feat_matrix[user_item_indices].multiply(user_item_ratings[:]).mean(axis=0)
    user_profile = user_item_ratings[:] * feat_matrix[user_item_indices]
    user_profile = user_item_ratings @ feat_matrix[user_item_indices]
    return user_profile[None, :]

def recommend_items_user_profile(testset, feat_matrix, top_n=10):
    recommendations = {}
    # Iterate through each user in the test set
    # for user_id, _, true_rating in testset:
    #     recommendations[user_id] = []
    for index, user_item in enumerate(tqdm(testset, desc="Processing items")):
        user_id, _, true_rating  = user_item
        user_profile = get_user_profile(user_id, feat_matrix) 
        # Compute similarity between user profile and all item profiles
        user_sim_scores = linear_kernel(user_profile, feat_matrix).flatten()
         # Get top N item indices
        top_item_indices = user_sim_scores.argsort()[-top_n:][::-1]
        # Map indices back to item_ids
        top_item_ids = meta_Appliances_new_reset['parent_asin'].iloc[top_item_indices].values
        # meta_Appliances_new_reset
        # top_item_ids = train_df['parent_asin'].iloc[top_item_indices].values
        # Get the top N recommended items
        # recommendations[user_id].extend(list(top_item_ids))
        recommendations[user_id] = list(top_item_ids)
    return recommendations
#  func test example
user_profile = get_user_profile('AFTEMMCIMU62ONZHLLXEXSME4Z6Q', tfidf_matrix)
user_profile

In [ ]:
contentbased_recom_result_pth = "/kaggle/input/meta-appliances-jsonl/top_n_recommendations_contentbased4.json"

if os.path.exists(contentbased_recom_result_pth):
    top_n_recommendations_contentbased = pd.read_json(contentbased_recom_result_pth)
else:
    top_n_recommendations_contentbased = recommend_items_user_profile(testset, tfidf_matrix, top_n=TOP_K)
    with open('/kaggle/working/top_n_recommendations_contentbased4.json', 'w') as f:
        json.dump(top_n_recommendations_contentbased, f)

In [ ]:
# Calculate precision, recall, and F1-score at K=10
eva_top_k = 100
avg_precision_contentbased, avg_recall_contentbased, avg_f1_score_contentbased, avg_ndg_contentbased = precision_recall_f1_at_k(top_n_recommendations_contentbased, testset, k=eva_top_k)
# RMSE = accuracy.rmse(recommendations)
# print("RMSE for content-based filtering: {}".format(RMSE))

print(f"Average Precision at K=3: {avg_precision_contentbased:.4f}")
print(f"Average Recall at K=3: {avg_recall_contentbased:.4f}")
print(f"Average F1-Score at K=3: {avg_f1_score_contentbased:.4f}")
print(f"Average NDG at K=3: {avg_ndg_contentbased:.4f}")

# hybrid mothods

In [ ]:
# Mixed Hybrid: This approach presents recommendations from different systems side by side. For instance, 
# you might display content-based recommendations alongside 
# collaborative filtering recommendations on the same page.

In [ ]:
# hybrid_recommendation = top_n_recommendations_contentbased + top_n_recommendations_svd + top_n_recommendations_itembased + top_n_recommendations_userbased
hybrid_recommendation = {}
# Iterate through each user in the test set
cat_top_k = 100
for user_id, _, true_rating in testset:
    hybrid_recommendation[user_id] = list(np.concatenate([top_n_recommendations_contentbased[user_id][:cat_top_k], top_n_recommendations_svd[user_id][:cat_top_k], \
                            top_n_recommendations_itembased[user_id][:cat_top_k], top_n_recommendations_userbased[user_id][:cat_top_k]]))
    # hybrid_recommendation[user_id] = np.concatenate([top_n_recommendations_contentbased[user_id][:cat_top_k]])

In [ ]:
# AFTEMMCIMU62ONZHLLXEXSME4Z6Q
ls1 = list(hybrid_recommendation['AFTEMMCIMU62ONZHLLXEXSME4Z6Q'])[:100]
ls2 = top_n_recommendations_contentbased['AFTEMMCIMU62ONZHLLXEXSME4Z6Q'][:100]
re1 = set(ls1)-set(ls2)
re1

In [ ]:
eva_top_k = 100
avg_precision_hybrid1, avg_recall_hybrid1, avg_f1_score_hybrid1, avg_ndg_hybrid1 = precision_recall_f1_at_k(hybrid_recommendation, testset, k=eva_top_k)

print(f"Average Precision at K=3: {avg_precision_hybrid1:.4f}")
print(f"Average Recall at K=3: {avg_recall_hybrid1:.4f}")
print(f"Average F1-Score at K=3: {avg_f1_score_hybrid1:.4f}")
print(f"Average NDG at K=3: {avg_ndg_hybrid1:.4f}")

In [ ]:
len(top_n_recommendations_userbased)
len(top_n_recommendations_itembased)
len(top_n_recommendations_svd)
len(top_n_recommendations_contentbased)
len(hybrid_recommendation)
len(hybrid_recommendation['AFTEMMCIMU62ONZHLLXEXSME4Z6Q'])
np.unique(hybrid_recommendation['AFTEMMCIMU62ONZHLLXEXSME4Z6Q']).shape
# hybrid_recommendation_pd = pd.DataFrame(hybrid_recommendation)
# hybrid_recommendation_pd.head(2)

In [ ]:
len(testset)
len(test_df['user_id'])
len((hybrid_recommendation))
len(top_n_recommendations_userbased.keys())

In [ ]:
from collections import Counter
final_recommendation = {}
for user_id, recommended_items in hybrid_recommendation.items():
    count_dict = Counter(recommended_items)
    sorted_items = sorted(count_dict.items(), key=lambda x: x[1], reverse=True)
    final_recommendation[user_id]  = [item[0] for item in sorted_items]

ls = hybrid_recommendation['AFTEMMCIMU62ONZHLLXEXSME4Z6Q']
count_dict = Counter(hybrid_recommendation['AFTEMMCIMU62ONZHLLXEXSME4Z6Q'])
sorted_items = sorted(count_dict.items(), key=lambda x: x[1], reverse=True)
ls[:10]
ls2 = [item[0] for item in sorted_items[:10]]
# sorted_items
len(ls2)
ls2
len(sorted_items)

In [ ]:
# eva_top_k = 100
avg_precision_hybrid_vote, avg_recall_hybrid_vote, avg_f1_score_hybrid_vote, avg_ndg_hybrid_vote = precision_recall_f1_at_k(final_recommendation, testset, k=eva_top_k)

print(f"Average Precision at K=3: {avg_precision_hybrid_vote:.4f}")
print(f"Average Recall at K=3: {avg_recall_hybrid_vote:.4f}")
print(f"Average F1-Score at K=3: {avg_f1_score_hybrid_vote:.4f}")
print(f"Average NDG at K=3: {avg_ndg_hybrid_vote:.4f}")

In [ ]:

def recommend_items_rank(hybrid_recommendation, feat_matrix, top_n=10):
    recommendation_scores = hybrid_recommendation
    for index, user_item in enumerate(tqdm(hybrid_recommendation.items(), desc="Processing items")):
        user_id, recom_seq  = user_item

        user_interactions = test_dict_user_purchased[user_id]
        user_item_indices = [item_indices[item_id] for item_id in user_interactions['item_id']]
        user_item_ratings = user_interactions['rating']
        user_item_feats    = feat_matrix[user_item_indices]
        recom_item_indices = []
        for recom_item in recom_seq:
            recom_item_indices.append(item_indices[recom_item])
        
        recom_item_feats = feat_matrix[recom_item_indices]

        # print(recom_item_feats.shape)
        # print(np.transpose(user_item_feats).shape)
        recom_scores = recom_item_feats @ np.transpose(user_item_feats)
        recom_scores = recom_scores @ user_item_ratings
        top_item_indices = recom_scores.argsort()[-top_n:][::-1]
        # print(len(top_item_indices))
        top_items = [hybrid_recommendation[user_id][i] for i in top_item_indices]
        # print(recom_scores)
        # print(top_items)
        # print(hybrid_recommendation[user_id][:top_n] )
        recommendation_scores[user_id][:top_n] = top_items
    
    return recommendation_scores
#  func test example
# hybrid_recommendation_ls1 = {}
# hybrid_recommendation_ls1['AFTEMMCIMU62ONZHLLXEXSME4Z6Q'] = hybrid_recommendation['AFTEMMCIMU62ONZHLLXEXSME4Z6Q']
# user_profile = recommend_items_rank(hybrid_recommendation_ls1, tfidf_matrix)
# user_profile

In [ ]:
hybrid_recommendation2 = recommend_items_rank(hybrid_recommendation, tfidf_matrix, top_n=100)

In [ ]:
eva_top_k = 50
avg_precision_hybrid2, avg_recall_hybrid2, avg_f1_score_hybrid2, avg_ndg_hybrid2 = precision_recall_f1_at_k(hybrid_recommendation2, testset, k=eva_top_k)

print(f"Average Precision at K=3: {avg_precision_hybrid2:.4f}")
print(f"Average Recall at K=3: {avg_recall_hybrid2:.4f}")
print(f"Average F1-Score at K=3: {avg_f1_score_hybrid2:.4f}")
print(f"Average NDG at K=3: {avg_ndg_hybrid2:.4f}")

In [ ]:
final_recommendation['AFTEMMCIMU62ONZHLLXEXSME4Z6Q'][:5]
hybrid_recommendation['AFTEMMCIMU62ONZHLLXEXSME4Z6Q'][:5]
# list(user_item_counts['AFTEMMCIMU62ONZHLLXEXSME4Z6Q'].keys())[:10]

In [ ]:

weights = {
    'content': 60,  
    'svd': 20,      
    'item': 10,     
    'user': 10      
}

cat_top_k = 400  


hybrid_recommendation = {}


for user_id, _, true_rating in testset:
    # for weight
    content_rec = top_n_recommendations_contentbased[user_id][:weights['content']]
    svd_rec = top_n_recommendations_svd[user_id][:weights['svd']]
    item_rec = top_n_recommendations_itembased[user_id][:weights['item']]
    user_rec = top_n_recommendations_userbased[user_id][:weights['user']]
    
    combined_rec = np.concatenate([content_rec, svd_rec, item_rec, user_rec])
    
    # 去重
    combined_rec = list(dict.fromkeys(combined_rec))  
    
    hybrid_recommendation[user_id] = combined_rec[:cat_top_k]

avg_precision_hybrid, avg_recall_hybrid, avg_f1_score_hybrid, avg_ndcg_hybrid = precision_recall_f1_at_k(
    hybrid_recommendation, testset, k=cat_top_k)

print(f"Average Precision at K={cat_top_k}: {avg_precision_hybrid:.4f}")
print(f"Average Recall at K={cat_top_k}: {avg_recall_hybrid:.4f}")
print(f"Average F1-Score at K={cat_top_k}: {avg_f1_score_hybrid:.4f}")
print(f"Average NDG at K={cat_top_k}: {avg_ndcg_hybrid:.4f}")



In [ ]:
# import torch
# from transformers import AutoModel, AutoTokenizer, TFBertModel
# from transformers import BertTokenizer, BertModel, BertConfig
# from sklearn.metrics.pairwise import cosine_similarity
# import transformers
# # from transformers import AutoTokenizer,TFBertModel
# save_directory='/kaggle/working/'
# print(transformers.__version__)

# model_checkpoint = 'bert-base-uncased'
# mode1_checkpt1 = '/kaggle/input/bert-base-cased-for-torch/pytorch/default/1/bert-base-cased-Torch'
# tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
# transformer_model = TFBertModel.from_pretrained(model_checkpoint)

# def encode_text(text, tokenizer):
    
#     encoded = tokenizer.batch_encode_plus(
#         text,
#         add_special_tokens=True,
#         max_length=50,
#         padding='max_length',
#         truncation=True,
#         return_attention_mask=True,
#         return_tensors="tf",
#     )

#     input_ids = np.array(encoded["input_ids"], dtype="int32")
#     attention_masks = np.array(encoded["attention_mask"], dtype="int32")

#     return {
#         "input_ids": input_ids,
#         "attention_masks": attention_masks
#     }





In [ ]:
# meta_Appliances_new.head(2)
# meta_Appliances_new['description_len'] = meta_Appliances_new['description'].apply(len)
# meta_Appliances_new.describe()['description_len']

In [ ]:
# # Appliances_review_meta_merge['combined_user_feat'] = Appliances_review_meta_merge.apply(lambda row: row['title_x']    + ', ' + row['text'], axis=1)
# # meta_Appliances_new['combined_item_feat'] = meta_Appliances_new.apply(lambda row: row['categories'] + ', ' + row['details'] + ', ' + row['title']  + ', ' + row['description'], axis=1)
# meta_Appliances_new['combined_item_feat'] = meta_Appliances_new.apply(
#     lambda row: ', '.join(row['categories']) + ', ' + ', '.join(row['details']) + ', ' + row['title'] + ', ' + ', '.join(row['description']),
#     axis=1)
# # meta_Appliances_new.apply( lambda row: ', '.join(row['description']), axis=1)
# meta_Appliances_new.head(2)
# # meta_Appliances_new['new1']

In [ ]:
# total_meta_encoder = encode_text(meta_Appliances_new['combined_item_feat'].tolist(), tokenizer)

In [ ]:
# tfidf_matrix.shape
# len(meta_Appliances_new['parent_asin'])
# len(X1_train['input_ids']

In [ ]:
# import tensorflow as tf
# # meta_Appliances_new.index = range(0, len(meta_Appliances_new))
# # item_indices = pd.Series(meta_Appliances_new.index, index=meta_Appliances_new['parent_asin']).to_dict()
# product_bert_embedding_list = []
# for index in tqdm(range(len(total_meta_encoder['input_ids']))):
#     # Extract the input IDs and attention masks for the first and second inputs
#     input_ids1 = tf.convert_to_tensor(total_meta_encoder['input_ids'][index])  # Convert to tensor
#     attention_mask1 = tf.convert_to_tensor(total_meta_encoder['attention_masks'][index])
#     # Add batch dimension for model infer
#     input_ids1 = tf.expand_dims(input_ids1, 0)  # Shape: (1, sequence_length)
#     attention_mask1 = tf.expand_dims(attention_mask1, 0)
#     # Get the output from the transformer model
#     output1 = transformer_model(input_ids=input_ids1, attention_mask=attention_mask1)
#     embedding_layer1 = output1.last_hidden_state[:, 0, :] # output1.last_hidden_state.mean(dim=1) / two different way to embedding
#     embedding_layer1 = tf.linalg.l2_normalize(embedding_layer1, axis=-1)
#     product_embedding_list.append(embedding_layer1)
# product_bert_embedding_list = np.array(product_embedding_list)

# product_embedding_list.shape

In [ ]:
# product_embedding_list = np.squeeze(product_embedding_list, axis=1)

In [ ]:
# contentbased_recom_result_pth = "/kaggle/input/meta-appliances-jsonl/top_n_recommendations_bert_contentbased.json"

# if os.path.exists(contentbased_recom_result_pth):
#     top_n_recommendations_bert_contentbased = pd.read_json(contentbased_recom_result_pth)
# else:
#     top_n_recommendations_bert_contentbased = recommend_items_user_profile(testset, product_embedding_list, top_n=TOP_K)
#     with open('/kaggle/working/top_n_recommendations_bert_contentbased.json', 'w') as f:
#         json.dump(top_n_recommendations_contentbased, f)

In [ ]:
# import torch
# import torch.nn as nn

# class RatingPredictionModel(nn.Module):
#     def __init__(self, bert_model):
#         super(RatingPredictionModel, self).__init__()
#         self.bert_model = bert_model
#         self.fc = nn.Linear(bert_model.config.hidden_size, 1)  # Regression layer for rating prediction
    
#     def forward(self, input_ids, attention_mask):
#         # Get BERT embeddings
#         outputs = self.bert_model(input_ids=input_ids, attention_mask=attention_mask)
#         # Use [CLS] token embedding
#         cls_embedding = outputs.last_hidden_state[:, 0, :]
#         # Predict rating
#         rating = self.fc(cls_embedding)
#         return rating